# Part V — APIs with Django (Django REST Framework + Beyond)  
## 23. API Fundamentals (Design, HTTP Semantics, Errors, Pagination, Versioning)

This chapter is “API design literacy.” Before using Django REST Framework (DRF),
you must understand what a good API *is* and why it’s structured the way it is.

A strong API foundation prevents:
- inconsistent endpoints across your team
- confusing error formats
- security mistakes (CSRF/CORS/auth)
- performance mistakes (no pagination, expensive list endpoints)
- breaking changes without versioning strategy

---

## 23.0 Learning Outcomes

By the end of this chapter, you should be able to:

1. Explain what an API is and how it differs from server-rendered HTML.
2. Design resource-oriented URLs and choose correct HTTP methods.
3. Use correct status codes and response bodies for success and errors.
4. Design consistent JSON shapes and naming conventions.
5. Implement pagination, filtering, sorting, and searching via query params (and
   explain the tradeoffs).
6. Understand API authentication options (session vs token vs JWT) and the CSRF/CORS
   implications.
7. Choose an API versioning strategy and apply it.
8. Write a basic JSON endpoint in Django (without DRF) to practice the concepts.
9. Produce an “API contract” for an endpoint: request, response, errors.

---

## 23.1 What an API Is (and Why You Need One)

An **API (Application Programming Interface)** in web context is a set of HTTP
endpoints designed primarily for **machines** (frontends, mobile apps, other
services) rather than human browsers.

### 23.1.1 HTML pages vs JSON APIs

#### Server-rendered HTML endpoint
- Client: browser
- Response: HTML
- Goal: display UI

Example:
- `GET /articles/hello-django/` → HTML page

#### JSON API endpoint
- Client: frontend JS, mobile app, another server, CLI script
- Response: JSON
- Goal: exchange structured data

Example:
- `GET /api/articles/42/` → JSON `{id, title, ...}`

### 23.1.2 One product can (and often should) have both
Common real-world architectures:
- Django templates for admin/internal pages
- JSON APIs for mobile apps and SPAs
- Webhooks endpoints for third-party integrations

---

## 23.2 REST (Practical, Not Religious)

REST is a style for designing APIs around **resources** and using HTTP methods
consistently.

### 23.2.1 Core REST concepts (what matters)
- **Resource**: a thing you manage (article, task, comment, user).
- **Collection**: group of resources (`/api/articles/`).
- **Representation**: how a resource is serialized (JSON).
- **Uniform interface**: consistent use of methods + status codes.

### 23.2.2 Resource-oriented URL design

Good patterns (nouns, collections):
- `/api/articles/` (collection)
- `/api/articles/42/` (single resource)
- `/api/tasks/` and `/api/tasks/123/`

Avoid “RPC-like” endpoints unless necessary:
- `/api/doPublishArticle/` (harder to standardize and document)

Sometimes action endpoints are fine if they represent a subresource or state change:
- `/api/articles/42/publish/` (action)
But you should treat it as a deliberate choice, not default.

### 23.2.3 Plural vs singular
Industry convention:
- collections are plural: `/api/articles/`
- single resource: `/api/articles/{id}/`

Consistency matters more than the exact preference.

---

## 23.3 HTTP Methods (Correct Semantics = Correct APIs)

### 23.3.1 Safe vs idempotent (critical for APIs)

- **Safe**: should not change server state (GET, HEAD).
- **Idempotent**: doing it multiple times has the same effect as once (PUT, DELETE
  typically; GET too).

Why you care:
- clients retry requests
- proxies may retry
- mobile networks are unreliable
- idempotency prevents duplicate creates/charges

### 23.3.2 CRUD mapping (typical)
- `GET /api/articles/` → list
- `POST /api/articles/` → create
- `GET /api/articles/{id}/` → retrieve
- `PUT /api/articles/{id}/` → replace
- `PATCH /api/articles/{id}/` → partial update
- `DELETE /api/articles/{id}/` → delete

### 23.3.3 PUT vs PATCH (how to choose)

- **PUT**: “replace the entire resource representation”
  - client sends full object
  - missing fields are removed/reset (conceptually)
- **PATCH**: “update only the fields provided”
  - client sends partial object
  - missing fields stay unchanged

Industry reality:
- most clients prefer PATCH for updates because they often only change a few fields

---

## 23.4 Status Codes (API Meaning + Example Payloads)

Status codes are part of your contract with clients.

### 23.4.1 Common API responses

#### 200 OK
Success with response body:
- GET returns data
- PATCH returns updated resource

#### 201 Created
Resource created (often returns created resource):
- POST create

Often includes `Location` header:
- `Location: /api/articles/42/`

#### 204 No Content
Success with no body:
- DELETE success
- or sometimes PATCH success without returning representation

#### 400 Bad Request
Client sent invalid data (validation errors):
- missing required fields
- invalid types
- invalid query params

#### 401 Unauthorized
Not authenticated:
- missing/invalid auth credentials

#### 403 Forbidden
Authenticated but not allowed:
- permission denied
- object-level restriction

#### 404 Not Found
Resource doesn’t exist OR you choose to hide it for authorization reasons.

#### 409 Conflict
Conflicts with current state:
- duplicate unique fields
- state transition invalid (e.g., “already published”)
Not mandatory, but useful.

#### 422 Unprocessable Entity (sometimes used)
Some APIs use 422 for validation errors. Django/DRF typically uses 400 for validation
errors. Choose one approach and be consistent.

---

## 23.5 JSON Shapes: Consistency Rules That Make APIs Pleasant

A “good” API is consistent even when many developers contribute.

### 23.5.1 Naming conventions: snake_case vs camelCase
Django + Python code typically uses `snake_case`. Many JS APIs use `camelCase`.

Industry approach options:
- Use `snake_case` in JSON (common with DRF; simplest for backend teams)
- Use `camelCase` in JSON (common with JS-heavy teams; requires conversion layer)

Choose one early and document it.

For this workbook, we’ll use `snake_case`.

### 23.5.2 Envelope vs “bare object”
Two patterns:

**Bare object**
```json
{ "id": 42, "title": "Hello" }
```

**Envelope**
```json
{ "data": { "id": 42, "title": "Hello" } }
```

For list responses, it’s common to use an envelope:

```json
{
  "count": 123,
  "next": "/api/articles/?page=2",
  "previous": null,
  "results": [ ... ]
}
```

This is common because it supports pagination metadata cleanly.

### 23.5.3 Error format (do not improvise per endpoint)

A consistent error format is one of the most important “industry standard” traits.

A practical pattern:

```json
{
  "error": {
    "code": "validation_error",
    "message": "Your request is invalid.",
    "details": {
      "title": ["This field is required."],
      "page": ["Must be >= 1."]
    }
  }
}
```

Or a DRF-like default:

```json
{
  "title": ["This field is required."],
  "page": ["A valid integer is required."]
}
```

Pick a shape and keep it consistent. DRF provides a default error structure; many
teams keep it rather than inventing a new one.

---

## 23.6 Content Negotiation: `Accept` and `Content-Type`

### 23.6.1 `Content-Type` (what you sent)
When a client sends JSON:

```http
Content-Type: application/json
```

When a client sends form data:

```http
Content-Type: application/x-www-form-urlencoded
```

### 23.6.2 `Accept` (what you want back)
Client asks for JSON:

```http
Accept: application/json
```

In practice, most JSON APIs always return JSON, and Accept is less used, but it
still matters when you support multiple formats (or when clients misbehave).

---

## 23.7 Pagination (Why It’s Query-Based, Patterns, Limitations)

Pagination is essential for:
- performance (don’t return 50,000 rows)
- reliability (avoid timeouts)
- UX (app can load incrementally)

### 23.7.1 Why query params like `?page=2` exist
Because pagination modifies “how you view the same collection.” The resource is
still `/api/articles/` but you’re asking for a specific slice.

- `/api/articles/?page=1`
- `/api/articles/?page=2`

### 23.7.2 Common pagination styles

#### Style A: Page number + page size
```
GET /api/articles/?page=2&page_size=50
```

Pros:
- user-friendly
- easy to implement
- easy “jump to page”

Cons:
- slow large offsets
- data shifting can cause duplicates/missing items

#### Style B: Limit + offset
```
GET /api/articles/?limit=50&offset=50
```

Pros:
- flexible for infinite scroll
- explicit “skip N, take M”

Cons:
- same offset issues as page numbers

#### Style C: Cursor pagination (more scalable)
```
GET /api/articles/?cursor=eyJpZCI6NDJ9&page_size=50
```

Pros:
- stable under inserts (when designed well)
- often faster for deep pagination

Cons:
- harder to debug
- not easy “jump to page 50”
- requires stable ordering

### 23.7.3 Industry rules for pagination
- Always cap page size (DoS prevention).
- Always define a stable ordering (e.g., `-created_at, -id`).
- Never return unpaginated large lists by default.

---

## 23.8 Filtering, Sorting, Searching (Query Params as a Standard Interface)

### 23.8.1 Filtering
```
GET /api/tasks/?status=open&priority=3
```

Rules:
- Use exact matches for most filters.
- Document allowed values.
- Validate and return 400 on invalid values.

### 23.8.2 Sorting
Common patterns:

- `?sort=created_at&direction=desc`
- or `?ordering=-created_at` (popular in DRF)

Sorting must be whitelisted. Never allow arbitrary SQL ordering from user input.

### 23.8.3 Searching
```
GET /api/articles/?q=django
```

Keep search distinct from filters.
For serious apps, search often becomes:
- PostgreSQL full-text search
- Elastic/OpenSearch
But basic icontains search is fine early.

---

## 23.9 Versioning (How to Change APIs Without Breaking Clients)

Clients depend on your API contract. If you change it, you can break mobile apps in
the field for months.

### 23.9.1 Common versioning strategies

#### Strategy A: URL path versioning (most common)
- `/api/v1/articles/`
- `/api/v2/articles/`

Pros:
- explicit and easy
- simple routing and docs
Cons:
- URL changes and duplication

#### Strategy B: Header versioning
- `Accept: application/vnd.company+json; version=1`

Pros:
- cleaner URLs
Cons:
- harder to debug manually

#### Strategy C: Query param versioning
- `?version=1`

Pros:
- easy to test
Cons:
- less standard; can be messy

Industry recommendation for most Django teams:
- use path versioning unless you have a strong reason not to

### 23.9.2 Compatibility rules (practical)
- Backward compatible changes:
  - add new optional fields
  - add new endpoints
- Breaking changes (require version bump):
  - remove/rename fields
  - change meaning of a field
  - change error shapes
  - change authentication expectations

---

## 23.10 Authentication for APIs (and CSRF/CORS Implications)

This is where many teams make mistakes.

### 23.10.1 Session authentication (browser + Django)
- Uses cookies (`sessionid`)
- Works great for server-rendered pages and same-site JS
- Requires CSRF protection for unsafe methods

If your API is consumed by:
- same site (same origin) pages with cookies
then session auth is fine.

### 23.10.2 Token authentication (common for APIs)
- Client sends token in header:
  - `Authorization: Token abc...`
  - or `Authorization: Bearer ...`
- No CSRF required for token auth (because browser doesn’t attach token automatically)
- Requires secure storage and rotation strategy

### 23.10.3 JWT (common but not always necessary)
JWTs are bearer tokens with claims.

Pros:
- stateless verification
- good for distributed systems

Cons:
- token revocation complexity
- security pitfalls (long-lived tokens, storing in localStorage, etc.)

**Industry guidance:**
- don’t choose JWT just because it’s popular
- choose based on client needs and security requirements

### 23.10.4 CORS (when frontend is on a different domain)
If your API is on `api.example.com` and frontend is on `app.example.com`, you must
handle CORS. This is separate from CSRF.

- CORS controls whether browsers allow JS to read responses across origins.
- CSRF controls whether cookie-auth requests are allowed to change state safely.

When you later build DRF APIs, we’ll configure these carefully.

---

## 23.11 Idempotency Keys (Advanced but Very Real for Payments/Creates)

For endpoints that create resources and may be retried (mobile networks!), you often
need idempotency.

Pattern:
- client sends an `Idempotency-Key` header on POST
- server stores result for that key for some time
- retry returns the same result instead of creating duplicates

Example:

```http
POST /api/payments/
Idempotency-Key: 7e2c5c1b...
```

This is critical for:
- payment charges
- order creation
- invitations

We won’t implement it now, but you should know it exists.

---

# 23.12 Hands‑On Lab (Without DRF): Build a Mini JSON API Properly

This lab is to practice fundamentals (methods, status codes, validation, pagination)
using plain Django.

We’ll create:
- `GET /api/articles/` list with `q`, `tag`, `page`, `page_size`
- `GET /api/articles/<slug>/` detail
- consistent error responses

## 23.12.1 Create a new app (optional) or add API views under articles
We’ll add a small `api` module inside `articles`:

Create `articles/api_views.py`:

```python
from __future__ import annotations

from django.core.paginator import EmptyPage, PageNotAnInteger, Paginator
from django.db.models import Q
from django.http import JsonResponse
from django.views.decorators.http import require_GET

from articles.models import Article


def error_response(*, code: str, message: str, details=None, status: int = 400):
    return JsonResponse(
        {
            "error": {
                "code": code,
                "message": message,
                "details": details or {},
            }
        },
        status=status,
    )


@require_GET
def api_article_list(request):
    q = (request.GET.get("q") or "").strip()
    tag = (request.GET.get("tag") or "").strip()

    page_raw = request.GET.get("page", "1")
    page_size_raw = request.GET.get("page_size", "20")

    try:
        page = int(page_raw)
        page_size = int(page_size_raw)
    except ValueError:
        return error_response(
            code="validation_error",
            message="page and page_size must be integers.",
            details={"page": page_raw, "page_size": page_size_raw},
            status=400,
        )

    if page < 1:
        return error_response(
            code="validation_error",
            message="page must be >= 1.",
            details={"page": page},
            status=400,
        )

    if page_size < 1:
        return error_response(
            code="validation_error",
            message="page_size must be >= 1.",
            details={"page_size": page_size},
            status=400,
        )

    if page_size > 100:
        page_size = 100

    qs = Article.objects.published().select_related("author").prefetch_related("tags")

    if tag:
        qs = qs.filter(tags__slug=tag)

    if q:
        qs = qs.filter(Q(title__icontains=q) | Q(body__icontains=q))

    qs = qs.distinct().order_by("-published_at", "-created_at")

    paginator = Paginator(qs, page_size)

    try:
        page_obj = paginator.page(page)
    except PageNotAnInteger:
        page_obj = paginator.page(1)
    except EmptyPage:
        page_obj = paginator.page(paginator.num_pages)

    results = []
    for a in page_obj.object_list:
        results.append(
            {
                "id": a.id,
                "slug": a.slug,
                "title": a.title,
                "status": a.status,
                "published_at": a.published_at.isoformat()
                if a.published_at
                else None,
                "author": {
                    "id": a.author_id,
                    "username": a.author.username,
                },
                "tags": list(a.tags.values_list("slug", flat=True)),
            }
        )

    def build_page_url(n: int | None):
        if n is None:
            return None
        params = request.GET.copy()
        params["page"] = str(n)
        params["page_size"] = str(page_size)
        return f"{request.path}?{params.urlencode()}"

    return JsonResponse(
        {
            "count": paginator.count,
            "page": page_obj.number,
            "page_size": page_size,
            "num_pages": paginator.num_pages,
            "next": build_page_url(page_obj.next_page_number())
            if page_obj.has_next()
            else None,
            "previous": build_page_url(page_obj.previous_page_number())
            if page_obj.has_previous()
            else None,
            "results": results,
        }
    )


@require_GET
def api_article_detail(request, slug: str):
    try:
        a = (
            Article.objects.published()
            .select_related("author")
            .prefetch_related("tags")
            .get(slug=slug)
        )
    except Article.DoesNotExist:
        return error_response(
            code="not_found",
            message="Article not found.",
            status=404,
        )

    return JsonResponse(
        {
            "id": a.id,
            "slug": a.slug,
            "title": a.title,
            "body": a.body,
            "published_at": a.published_at.isoformat()
            if a.published_at
            else None,
            "author": {"id": a.author_id, "username": a.author.username},
            "tags": [{"name": t.name, "slug": t.slug} for t in a.tags.all()],
        }
    )
```

### What you should learn from this lab code
- We validate query params and return structured 400 errors.
- We cap `page_size` to 100 (DoS protection).
- We return pagination metadata and URLs.
- We keep consistent JSON shapes for list vs detail.
- We use ORM optimization (`select_related`/`prefetch_related`) because APIs often
  serialize related data.

## 23.12.2 Wire API URLs
Create `articles/api_urls.py`:

```python
from django.urls import path

from . import api_views

app_name = "articles_api"

urlpatterns = [
    path("articles/", api_views.api_article_list, name="article_list"),
    path("articles/<slug:slug>/", api_views.api_article_detail, name="article_detail"),
]
```

Include them in `config/urls.py`:

```python
path("api/", include("articles.api_urls")),
```

Now you have:
- `GET /api/articles/`
- `GET /api/articles/<slug>/`

## 23.12.3 Test with curl (and observe status codes + payloads)

List:

```bash
curl -i "http://127.0.0.1:8000/api/articles/?page=1&page_size=5"
```

Invalid page:

```bash
curl -i "http://127.0.0.1:8000/api/articles/?page=abc"
```

Filter:

```bash
curl -i "http://127.0.0.1:8000/api/articles/?tag=django&q=templates"
```

Detail:

```bash
curl -i "http://127.0.0.1:8000/api/articles/hello-django/"
```

404:

```bash
curl -i "http://127.0.0.1:8000/api/articles/does-not-exist/"
```

---

## 23.13 Documentation (OpenAPI) — What “Industry Standard” Means

Most professional APIs have machine-readable docs, commonly **OpenAPI** (Swagger).

You should be able to describe each endpoint with:
- path + method
- query params
- request body schema (for POST/PATCH)
- response schema (success + errors)
- authentication requirements

In the next chapters, DRF tooling makes OpenAPI generation much easier.

---

## 23.14 Exercises (Do These Before Proceeding)

1. Design an API for tasks (org-scoped):
   - `GET /api/orgs/<org_slug>/tasks/`
   - filters: `status`, `assigned_to=me`, `q`
   - pagination: `page`, `page_size`
   - define response shape and error format

2. Add sorting to `/api/articles/`:
   - `?ordering=-published_at` or `?sort=published_at&direction=desc`
   - whitelist allowed fields
   - return 400 on invalid sort

3. Add a consistent “validation error details” structure:
   - return field-level messages in `error.details`
   - write 3 invalid requests and show their responses

4. Write a short API versioning decision:
   - choose `/api/v1/` vs header versioning
   - list 3 “breaking changes” that would require a new version

---

## 23.15 Chapter Summary

- Good APIs are consistent, resource-oriented, and use HTTP semantics correctly.
- Query params are the standard interface for pagination/filtering/sorting/search.
- Status codes + error formats are part of your public contract.
- Authentication choice (session/token/JWT) affects CSRF/CORS requirements.
- Versioning prevents breaking clients.
- You practiced fundamentals by building a mini JSON API without DRF.

---

Next chapter: **Part V — 24. Django REST Framework (DRF) Core**  
We’ll rebuild your API using DRF properly: serializers, viewsets, routers,
authentication/permissions, pagination, filtering, and tests—following
industry-standard DRF conventions.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../4. Professional_Django/22. architecture_and_code_organization.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='24. django_rest_framework.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
